In [ ]:
pip install setfit

In [ ]:
pip install spacy

In [ ]:
!spacy download en_core_web_sm

In [ ]:
# pip install -U  transformers

In [ ]:
# !nvidia-smi

In [ ]:
# chekc if cuda is available
# import torch
# torch.cuda.is_available()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ginkgogo/ca_restaurants_random_sample", split="train")

train_dataset = dataset.select(range(50))
eval_dataset = dataset.select(range(50, 102))

In [ ]:
train_dataset

In [ ]:
eval_dataset

In [ ]:
from setfit import AbsaModel

model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    spacy_model="en_core_web_sm",
)

In [ ]:
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset, AbsaTrainer
from transformers import EarlyStoppingCallback

args = TrainingArguments(
    output_dir="models",
    num_epochs=5,
    use_amp=True,
    batch_size=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)

trainer = AbsaTrainer(
    model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

In [ ]:
metrics = trainer.evaluate(eval_dataset)
print(metrics)

In [ ]:
pip install -U "huggingface_hub[cli]"

In [ ]:
!huggingface-cli login

In [ ]:
# model.save_pretrained("models/setfit-absa-model")
# or
# model.push_to_hub("ginkgogo/setfit-absa-bge-small-en-v1.5-restaurants")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!huggingface-cli login

In [ ]:
from setfit import AbsaModel

# Download from the 🤗 Hub
model = AbsaModel.from_pretrained(
    "ginkgogo/setfit-absa-bge-small-en-v1.5-restaurants-aspect",
    "ginkgogo/setfit-absa-bge-small-en-v1.5-restaurants-polarity",
    spacy_model="en_core_web_sm",
)
# Run inference
preds = model("The food was great, but the venue is just way too busy.")


In [ ]:
# print(preds)

In [ ]:


import pandas as pd

df = pd.read_csv(
  '/content/drive/MyDrive/699/ca_restaurants.csv'
)

bus_used_in_train = [234152, 88955, 174286, 228338, 203671, 151156, 88166, 64932, 142804, 210180, 35159, 90839, 137484, 85880, 128479, 92603, 20842, 200330, 175440, 8844, 61777, 3815, 123379, 125840, 180129, 206443, 219869, 101729, 107887, 188230, 244420, 49208, 139902, 242337, 35581, 228649, 44946, 32763, 69556, 152494, 5069963, 3915492, 4486491]

random_df_2000 = df.sample(2000)

for business in bus_used_in_train:
  if business in random_df_2000['business_id']:
    random_df_2000.drop(business, inplace=True)


print(random_df_2000.shape)

In [ ]:
sentences = list(random_df_2000['text'].str.lower())
# sentences
preds = model(sentences)

In [ ]:
print(preds)
print(len(preds))

In [ ]:
aspects_sentiment = []
for i in preds:
  if len(i) > 0:
    aspects_sentiment.append(i)
  else:
    aspects_sentiment.append('{}')

random_df_2000['aspects_sentiment'] = aspects_sentiment

In [ ]:
def extract(aspect_list):
  if isinstance(aspect_list, list):
    aspect_dict = {}
    for aspect in aspect_list:
      aspect_dict[aspect['span']] = aspect['polarity']
    return aspect_dict


In [ ]:
# for a in aspects_sentiment:
#   if len(a) > 1:
#     print(a)

In [ ]:
random_df_2000['aspects_sentiment'] = random_df_2000['aspects_sentiment'].apply(extract)

In [ ]:
# random_df_2000[random_df_2000['aspects_sentiment'] != None]


In [ ]:
random_df_2000.head()

In [ ]:
with_aspect_df = random_df_2000.dropna(subset=['aspects_sentiment'])
print(with_aspect_df.shape)

In [ ]:
random_df_2000['aspects_sentiment'] = random_df_2000['aspects_sentiment'].fillna('{}')

In [ ]:
random_df_2000.head()

In [ ]:
flatten_asepct = pd.json_normalize(random_df_2000['aspects_sentiment'])

In [ ]:
aspects = list(flatten_asepct.columns)

In [ ]:
flatten_asepct

In [ ]:
random_df_2000.reset_index(inplace=True)
flatten_asepct.reset_index(inplace=True)
final_df = pd.concat([random_df_2000, flatten_asepct], axis=1)
final_df.head()

In [ ]:
print(random_df_2000.shape)
print(flatten_asepct.shape)

In [ ]:
final_df.shape

In [ ]:
# len(final_df.business_id.unique())

In [ ]:
with_aspect_df.reset_index(inplace=True)
with_aspect_flatten.reset_index(inplace=True)
with_aspect_flatten = pd.json_normalize(with_aspect_df['aspects_sentiment'])
with_aspect_final_df = pd.concat([with_aspect_df, with_aspect_flatten], axis=1)

In [ ]:
with_aspect_flatten.shape

In [ ]:
with_aspect_df.shape

In [ ]:
with_aspect_final_df.shape

In [ ]:
with_aspect_final_df.head()

In [ ]:
# with_aspect_final_df.to_csv('with_aspect_from_random_2k.csv')
# !cp with_aspect_from_random_2k.csv '/content/drive/MyDrive/699/'

In [ ]:
# final_df.to_csv('random_2k.csv')
# !cp random_2k.csv '/content/drive/MyDrive/699/'

In [ ]:
import pandas as pd
out = pd.read_csv('/content/drive/MyDrive/699/with_aspect_from_random_2k.csv')
out.head()

In [ ]:
out_random = pd.read_csv('/content/drive/MyDrive/699/random_2k.csv')
out_random.shape